# Stacking Technique

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
pd.set_option('display.max_columns',None)
from sklearn.ensemble import VotingClassifier

C:\Users\Aman saini\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
dataset=pd.read_csv('F:/r file/data_cleaned.csv')

In [4]:
dataset

,Survived,Age,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,SibSp_0,SibSp_1,SibSp_2,SibSp_3,SibSp_4,SibSp_5,SibSp_8,Parch_0,Parch_1,Parch_2,Parch_3,Parch_4,Parch_5,Parch_6,Embarked_C,Embarked_Q,Embarked_S
0,0,22.000000,7.2500,0,0,1,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
1,1,38.000000,71.2833,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
2,1,26.000000,7.9250,0,0,1,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
3,1,35.000000,53.1000,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
4,0,35.000000,8.0500,0,0,1,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,27.000000,13.0000,0,1,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
887,1,19.000000,30.0000,1,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
888,0,29.699118,23.4500,0,0,1,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
889,1,26.000000,30.0000,1,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0


In [5]:
dataset.shape

(891, 25)

In [6]:
dataset.isnull().sum()

Survived      0
Age           0
Fare          0
Pclass_1      0
Pclass_2      0
Pclass_3      0
Sex_female    0
Sex_male      0
SibSp_0       0
SibSp_1       0
SibSp_2       0
SibSp_3       0
SibSp_4       0
SibSp_5       0
SibSp_8       0
Parch_0       0
Parch_1       0
Parch_2       0
Parch_3       0
Parch_4       0
Parch_5       0
Parch_6       0
Embarked_C    0
Embarked_Q    0
Embarked_S    0
dtype: int64

# Making the train test split

In [7]:
X=dataset.drop(['Survived'],axis=1)
Y=dataset['Survived']

In [8]:
from sklearn.model_selection import train_test_split as tts
X_train ,X_test,Y_train,Y_test=tts(X,Y,random_state=101,stratify=Y)

In [9]:
X_train.shape ,X_test.shape,Y_train.shape,Y_test.shape

((668, 24), (223, 24), (668,), (223,))

# Base Model

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

# Model Training And Prediction

In [11]:
def model_prediction(model,train_x,train_y,test_x):
    #train the model
    model.fit(train_x,train_y)
    
    #starting prediction
    pred_train=model.predict(train_x)
    pred_test=model.predict(test_x)
    
    return pred_train,pred_test

In [12]:
# Model1 -- Decision Tree
DT=DecisionTreeClassifier(random_state=101)
M1_train,M1_test = model_prediction(DT,X_train,Y_train,X_test)

As we very are using three base machine learning model 'Decision Tree','LogisticRegression','KNeighborsClassifier'.
Except Decsion tree rest two model are very well disturbed with non-scaling data which leads to high variance.
To overcome lets scale them first.

In [13]:
# Standard Scaler
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaler.fit(X=X_train)

X_train=scaler.transform(X_train)
X_test=scaler.transform(X_test)

In [14]:
#Model2 -- Logistic regression
LR=LogisticRegression(random_state=101)
M2_train,M2_test=model_prediction(LR,X_train,Y_train,X_test)

In [15]:
# Model3 -- Kneighbour
KN=KNeighborsClassifier()
M3_train,M3_test=model_prediction(KN,X_train,Y_train,X_test)

# Stacking Model

In [16]:
# Creating the new training Dataset
train_prediction={
        'DT':M1_train,
        'LR':M2_train,
        'KN':M3_train
}

test_prediction={
        'DT':M1_test,
        'LR':M2_test,
        'KN':M3_test
}       

train_prediction=pd.DataFrame(train_prediction)
test_prediction=pd.DataFrame(test_prediction)

In [18]:
train_prediction.head()

,DT,LR,KN
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0


In [19]:
test_prediction.head()

,DT,LR,KN
0,1,0,0
1,0,0,0
2,1,0,0
3,0,0,0
4,0,1,1


In [21]:
#stack model
model=LogisticRegression()
model.fit(train_prediction,Y_train)

LogisticRegression()

In [24]:
model.score(train_prediction,Y_train),model.score(test_prediction,Y_test)

(0.9880239520958084, 0.7533632286995515)

In [29]:
model.predict(train_prediction)[0:6]

array([0, 0, 0, 0, 0, 0], dtype=int64)

In [30]:
Y_train.head()

117    0
884    0
60     0
487    0
485    0
Name: Survived, dtype: int64